In [ ]:
# %pip install apache-airflow
# %pip install pendulum
# %pip install --upgrade pendulum

In [ ]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta

default_args = {
    'owner': 'admin',
    'depends_on_past': False,
    'start_date': datetime(2024, 1, 1),
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG(
    'ekstraksi_data_dag',
    default_args=default_args,
    description='DAG untuk mengekstraksi data buku dari API',
    schedule_interval='0 9 * * 1',  # Setiap Senin jam 09:00
    catchup=False
)

def extract_data():
    import requests
    url = "https://gist.githubusercontent.com/nadirbslmh/8922f71875948802321ef479a017f4c0/raw/1fbbc4b3d55f8ae717eb197d9aaf530ed1bc7ed2/sample.json"
    response = requests.get(url)
    data = response.json()
    relevant_books = []
    for book in data['data']:
        relevant_books.append({
            'judul': book['title'],
            'pengarang': book['author'],
            'tahun_terbit': book['year'],
            'genre': book['genre']
        })

    print(relevant_books)

ekstraksi_data_task = PythonOperator(
    task_id='ekstraksi_data',
    python_callable=extract_data,
    dag=dag,
)

dag
